# Generators and Coroutines

Generators allow you to declare a function that behaves like an iterator - a lazy iterator.
From https://realpython.com/introduction-to-python-generators/

Generators are for creating large lists that don't require large amount of memory like a list see PEP255. They behave like iterators but are implemented differently.

Generators produce data, coroutines consume data.

In [59]:
import random

def random_number_sequence(limit):
    """a generator to create a list of random numbers"""
    num = limit
    while num > 0:
        # use of 'yield' is what makes it a generator object rather than a standard object
        # state of generator function object saved here and number returned
        yield random.randint(0, 100)
        # start again from here when next() is called
        num -= 1

print("type(random_number_sequence) =", str(type(random_number_sequence))[1:-1])
for i in random_number_sequence(5):
    print(i)

type(random_number_sequence) = class 'function'
54
60
90
95
56


The generator object is an iterator so we can call `next()` on a generator object...

In [60]:
generator_fn = random_number_sequence(10)
print(next(generator_fn))
print(next(generator_fn))
print(next(generator_fn))

56
10
98


Rather than a function you can use a generator expression or comprehension...

In [61]:
generator_comprehension = (random.randint(0, 100) for x in range(0, 5))   # yields for each x
print("type(generator_comprehension) =", str(type(generator_comprehension))[1:-1])
for i in generator_comprehension:
    print(i)

type(generator_comprehension) = class 'generator'
26
18
85
88
80


You can use a generator to convert a string to hexidecimal

In [62]:
hex_generator = ("{:02x}".format(ord(c)) for c in "Hello")  # yields after each character in string
print(":".join(hex_generator))                              # join accepts an iterable

48:65:6c:6c:6f


Comparing a list comprehension with a generator comprehension...

In [63]:
nums_squared_lc = [num**2 for num in range(5)]
nums_squared_gc = (num**2 for num in range(5))
print("[num**2 for num in range(5)] =", nums_squared_lc)
print("(num**2 for num in range(5)) =", str(nums_squared_gc)[1:-1])

[num**2 for num in range(5)] = [0, 1, 4, 9, 16]
(num**2 for num in range(5)) = generator object <genexpr> at 0x00000228D1C03BC0


These do the same thing but have a different memory footprint. This creates a whole list through comprehension and then iterates.

In [64]:
for i in nums_squared_lc:          # all values stored before getting here
    print(i)
for i in nums_squared_gc:          # this creates data as it goes
    print(i)

0
1
4
9
16
0
1
4
9
16


`max()` also takes an iterable as a parameter so we can pass generator to max...

In [65]:
nums_squared_gc = (num**2 for num in range(5))    # need to recreate as generator already used
print(max(nums_squared_gc))

16


What if I want to return a value from a generator?

In [66]:
def single_shot() -> dict:              # generator returns a dictionary
    yield 'Fire!'
    return {'Success': True}            # returning value from generator function

my_single_shot = single_shot()
print(next(my_single_shot))
try:
    print(next(my_single_shot))
except StopIteration as e:              # StopIteration exception is raised
    print("StopIteration exception: generator finished and returned value:", e)

Fire!
StopIteration exception: generator finished and returned value: {'Success': True}


Will a generator comprehension raise an exception?

In [67]:
single_shot_comprehension = (random.randint(0, 100) for x in range(0, 1))
print(next(single_shot_comprehension))
try:
    print(next(single_shot_comprehension))
except StopIteration as e:              # StopIteration exception is raised again
    # ...but no return value to print
    print("StopIteration exception: generator finished and returned value:", e)

14
StopIteration exception: generator finished and returned value: 


What attributes does a generator differ from a standard function...

In [68]:
def single_shot_fn():
    return True

print("Difference in attributes between a generator function and standard function:",
      (set(dir(single_shot)) - set(dir(single_shot_fn))))
print("Difference in attributes between a generator object and standard function:",
      (set(dir(my_single_shot)) - set(dir(single_shot_fn))))
print("Difference in attributes between a generator object from a comprehension:",
      (set(dir(my_single_shot)) - set(dir(single_shot_comprehension))))

Difference in attributes between a generator function and standard function: set()
Difference in attributes between a generator object and standard function: {'gi_running', 'throw', '__next__', 'gi_code', 'close', 'gi_yieldfrom', '__del__', 'gi_frame', '__iter__', 'send'}
Difference in attributes between a generator object from a comprehension: set()


You can have multiple yields in a generator function...

In [69]:
def triple_shot():
    yield 'First yield!'
    yield 'Second yield!'
    yield 'Third yield!'

my_triple_generator = triple_shot()
print(next(my_triple_generator))
print(next(my_triple_generator))
print(next(my_triple_generator))

First yield!
Second yield!
Third yield!


Generators produce data like an iterable while with a coroutines it can consume data. First let's create a generator that uses `yield` as an expression rather than statement and make use of the return value from the first `yield`.

In [70]:
def my_coroutine_fn():
    print("my_coroutine_fn: Before first yield")          # note when this is printed
    ret = (yield "First yield from my_coroutine_fn")      # yield as expression and not statement
    print("my_coroutine_fn: After first yield, return value =", ret)
    if ret is None:
        yield "Second yield from my_coroutine_fn"
    else:
        yield "Different second yield!"

try:
    my_coroutine = my_coroutine_fn()
    print("try #1: Calling first next()")
    print("try #2:", next(my_coroutine))
    print("try #3:", next(my_coroutine))
    print("try #4:", next(my_coroutine))                  # this next() will throw as generator done
except StopIteration:
    print("except: Generator complete!")

try #1: Calling first next()
my_coroutine_fn: Before first yield
try #2: First yield from my_coroutine_fn
my_coroutine_fn: After first yield, return value = None
try #3: Second yield from my_coroutine_fn
except: Generator complete!


How do we send a return value back to the generator. We can use `send()`. This is what turns a generator into a coroutine, one that consumes data.

In [71]:
try:
    my_coroutine = my_coroutine_fn()
    print("try: Calling first next()")
    print("try #1:", next(my_coroutine))
    print("try #2: sending data to coroutine")
    print("try #3:", my_coroutine.send("Whatever you like"))    # send() is also a next()
    print("try #4:", next(my_coroutine))
except StopIteration:
    print("except: Generator complete!")

try: Calling first next()
my_coroutine_fn: Before first yield
try #1: First yield from my_coroutine_fn
try #2: sending data to coroutine
my_coroutine_fn: After first yield, return value = Whatever you like
try #3: Different second yield!
except: Generator complete!


Here is a complicated way to create a list using a co-routine

In [72]:
def my_number_swallower_fn():
    _mylist = []
    while len(_mylist) < 5:
        _mylist.append((yield))   # need to yield before you can do a send
    return _mylist

try:
    my_number_swallower = my_number_swallower_fn()
    next(my_number_swallower)        # need to get to first yield
    for i in range(100):
        my_number_swallower.send(i)  # co-routine is just consuming data
except StopIteration as e:
    print(e)

[0, 1, 2, 3, 4]


Call `throw()` on generator object...

In [73]:
def test_yield():
    try:
        yield 1
        yield 2
        yield 3
    except ValueError as e:
        print("test_yield: Caught value error exception in generator:", e)

my_throw_yield = test_yield()
value = next(my_throw_yield)
try:
    if value == 1:
        my_throw_yield.throw(ValueError("Don't like looks of this!"))
except StopIteration:
    print("except: Caught stop iteration - thrown by throw() function")

test_yield: Caught value error exception in generator: Don't like looks of this!
except: Caught stop iteration - thrown by throw() function


Call `close()` on generator object...

In [74]:
my_close_yield = test_yield()
value = next(my_close_yield)
if value == 1:
    my_close_yield.close()
try:
    # should be 2 if hadn't closed the generator
    value = next(my_close_yield)
except StopIteration:
    print("except: Caught stop iteration - thrown by next() function")

except: Caught stop iteration - thrown by next() function


`yield from` allows for delegation to sub-generators. See https://docs.python.org/3/whatsnew/3.3.html#pep-380.

In [75]:
my_sub_generator = ['Bob', 'Sally', 'Donald']       # a list can be considered a sub-generator

def my_generator():
    yield from my_sub_generator                     # this could be any generator function or comprehension
    yield from range(3)
    yield from (x for x in my_sub_generator)        # if I just yield, it will yield a generator expression

for x in my_generator():
    print(x)

Bob
Sally
Donald
0
1
2
Bob
Sally
Donald


Can be useful for recursive traversal of tree...

In [76]:
my_nested_list = [[[1, 2, 3], 'Bob', 'John'], ['Sally', 'Susan', 'Amelia']]

def traverse_nested_lists(a_node):
    if type(a_node) is not list:
        print(f"Yield a leaf {a_node}")
        yield a_node
    else:
        print(f"Recurse a list {a_node}")
        for node in a_node:
            yield from traverse_nested_lists(node)

for x in traverse_nested_lists(my_nested_list):
    pass

Recurse a list [[[1, 2, 3], 'Bob', 'John'], ['Sally', 'Susan', 'Amelia']]
Recurse a list [[1, 2, 3], 'Bob', 'John']
Recurse a list [1, 2, 3]
Yield a leaf 1
Yield a leaf 2
Yield a leaf 3
Yield a leaf Bob
Yield a leaf John
Recurse a list ['Sally', 'Susan', 'Amelia']
Yield a leaf Sally
Yield a leaf Susan
Yield a leaf Amelia
